<a href="https://colab.research.google.com/github/igedeaguss/Magang-Pharos/blob/main/Product%20Match%20Similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pymssql
!pip install rapidfuzz
import pymssql
import pandas as pd
import numpy as np
from google.colab import auth
from google.auth import default
import re
from rapidfuzz import fuzz, process
import warnings
warnings.filterwarnings("ignore")
from google.colab import drive
from google.colab import auth
auth.authenticate_user()
import gspread
from gspread_dataframe import get_as_dataframe
from google.auth import default

#Mount Google Drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.5 MB/s eta 0:00:00
Mounted at /content/drive


# Data

In [ ]:
connection = pymssql.connect(server='35.187.243.174', user='tim_powerbi',
                             password='dGltcG93ZXJiaQ', database='mkt_insight', port = '1433')

query_2 = """
SELECT [Kode Produk PI], [Nama Produk], Procode, Prodesc FROM mkt_insight.dbo.ListAllProductComplete
GROUP BY [Kode Produk PI], [Nama Produk], Procode, Prodesc"""

data_produk_pi = pd.DataFrame(pd.read_sql_query(query_2, connection))

data_produk_pi

,Kode Produk PI,Nama Produk,Procode,Prodesc
0,000058,FAKTU OINTMENT 20 G,0101183,FAKTU OINT 20GR
1,000069,FAKTU SUPPOSITORIA,0101184,FAKTU SUPP 10`S
2,000070,NEBACETIN OINTMENT 5 G,0102055,NEBACETIN OINT 5GR
3,000080,NEBACETIN POWDER 5 G,0102056,NEBACETIN POWDER 5GR
4,000102,ALBOTHYL GEL 50 G,0100061,ALBOTHYL 18MG GEL 50GR
...,...,...,...,...
1235,0514363,ILLUMINARE PORE CLEAR FACIAL WASH 100G,0514363,ILLUMINARE PORE CLEAR FACIAL WASH 100G
1236,3000854,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML,3000854,ILLUMINARE YOUTH ACTIVATOR NOURISH SERUM 30 ML
1237,3000855,ILLUMINARE PORE SERUM 30ML,3000855,ILLUMINARE PORE SERUM 30ML
1238,3002105,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...,3002105,ILLUMINARE SUN PROTECT FACE & BODY LOTION SPF5...


In [2]:
data_ims = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/IMS Compiled Q3 2019 - Q2 2024 (List Product).xlsx', sheet_name='Sheet1')
data_survey = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Hospital Survey Product Name Pivot.xlsx', sheet_name='Sheet1')
data_pi_2024_2025 = pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI Sales 2024 - 2025.xlsx', sheet_name='Sheet1', dtype={'Item Kode':str})

In [3]:
data_prdk_survey = data_survey.copy()
data_ims_prdk = data_ims.copy()
#data_pi_prdk = data_produk_pi[['Kode Produk PI', 'Nama Produk']].copy()
data_pi_prdk = data_pi_2024_2025.copy()

In [4]:
data_pi_prdk

,Item Kode,Nama Produk,Total Penjualan
0,009100,HERCUM 0.05 % CREAM 10 G,9.276000e+08
1,013852,ACUCEL 0.1% GEL 10 G,-8.800000e+04
2,008220,IFLACORT CREAM 5 G,4.093264e+08
3,027173,AYU DERMA WHITENING CARE DAY CREAM 30 G,2.411600e+07
4,029670,NOURISH ACNE PLAST GIRL PROMO BUNDLING,1.018624e+09
...,...,...,...
361,027536,SILKY WHITE WHITENING DAY CREAM 25 G,2.052610e+07
362,012917,QUINOBIOTIC 200 MG/100 ML INFUSION,7.063620e+08
363,004810,ANTIPRESTIN 20 KAPSUL,1.201197e+09
364,006108,TRICHOSTATIC OVULA,1.179210e+08


In [ ]:
data_prdk_survey

,Kode,Product Name
0,1,ATORVASTATIN 20MG TAB/AMAROX
1,2,BISOPROLOL 5 MG /HJ
2,3,BISOPROLOL 5 MG TABLET
3,4,CALOS KAPLET
4,5,CAPTOPRIL 12.5 MG
...,...,...
50330,50331,ZYVOX 600MG INJ 300ML
50331,50332,ZYVOX 600MG TAB
50332,50333,ZYVOX 600MG/300ML POLYBAG
50333,50334,Zyvox Infusion 2 mg/mL - 300 mL


# Cleaning Data

In [4]:
#Membersihkan nama brand IMS
def clean_product_name(name):
    return name.split(' -')[0].strip().lower() if isinstance(name, str) else ""

# Daftar bentuk sediaan yang diakui
BENTUK_SEDIAAN = [
    "TAB", "CAP", "SYRUP", "SYR", "INJ", "CREAM", "GEL", "SUSP", "EFF", "ODT",
    "PWD", "OINT", "SUPPOS", "INF", "DRP", "CONC", "SOL", "AMP", "ML", "MG", "IU", "MCG", "G"
]

def clean_product_name_survey(product_name):
    """
    Membersihkan nama produk dengan aturan:
    1. Menghapus produk jika mengandung kata 'racikan' atau 'racik'.
    2. Menghapus teks dalam tanda kurung, kecuali jika mengandung angka atau bentuk sediaan.
    3. Jika tanda kurung berisi angka atau bentuk sediaan, hapus tanda kurungnya tetapi biarkan isinya.
    """
    if not isinstance(product_name, str) or product_name.strip() == '':
        return None  # Abaikan jika input kosong atau bukan string

    product_name = product_name.strip()

    # Jika nama produk mengandung "racikan" atau "racik", maka diabaikan (return None)
    if re.search(r'\bracik(an)?\b', product_name, re.IGNORECASE):
        return None

    # Fungsi untuk memproses tanda kurung
    def process_parentheses(match):
        content = match.group(1).strip()
        contains_number = bool(re.search(r'\d', content))  # Cek apakah ada angka
        contains_form = any(sediaan in content.upper() for sediaan in BENTUK_SEDIAAN)  # Cek apakah ada bentuk sediaan

        if contains_number or contains_form:
            return f" {content} "  # Hilangkan tanda kurung, tetapi biarkan isinya
        return ''  # Hapus jika tidak mengandung angka atau bentuk sediaan

    # Hapus tanda kurung yang tidak mengandung angka/bentuk sediaan atau hilangkan kurungnya jika cocok
    cleaned_name = re.sub(r'\((.*?)\)', process_parentheses, product_name)

    # Hilangkan spasi ganda yang mungkin muncul
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()

    return cleaned_name


#Standarisasi bentuk sediaan
def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'TAB',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'CAP',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'CAP',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'SUSP',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'PWD',
        r'\boint\b|\bointment\b|\bsalep\b': 'OINT',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'CAP',
        r'\binj\b|\binjeksi\b|\binjection\b': 'INJ',
        r'\bamp\b|\bampule\b|\bampul\b': 'AMP',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'CRM',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'SUPPOS',
        r'\binf\b|\binfusion\b|\binfus\b': 'INF',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'SYR',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'DRP',
        r'\bconc\b|\bconcentrate\b': 'CONC',
        r'\bsol\b|\bsolution\b': 'SOL',
        r'\bgel\b': 'GEL'
    }

    if not isinstance(product_name, str) or product_name.strip() == '':
        return ''

    # Ubah bentuk sediaan dalam nama produk
    for pattern, replacement in dosage_form_map.items():
        product_name = re.sub(pattern, replacement, product_name, flags=re.IGNORECASE)

    return product_name  # Tidak langsung uppercase agar mudah dibaca dalam tahap sebelumnya

# Standarisasi: Ubah "syringe" menjadi "inj" dalam data IMS
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].str.replace(r"\bsyringe\b", "inj", regex=True)


# Daftar kata kunci alat kesehatan
medical_device_keywords = [
    "stop cock", "catheter", "stent", "scalpel", "forceps", "retractor",
    "bandage", "suture", "gloves", "gown", "dental", "lens", "surgical", "probe", "monitor",
    "electrode", "defibrillator", "ventilator", "prosthesis", "wire", "needle", "screw", "plate",
    "straw", "catether", "selang", "stopcock", "incisor", "sponge", "locking", "accu check", "accucheck",
    "arm sling", "action reliever", "active lite", "adult bacterial", "oxygenator", "adult rem",
    "advantime", "aesculap", "affinity", "alas", "alcohol swab", "allevyn adhesive","face mask", "GYNECOLOGY SET","ALERT SNAP"
]

# Fungsi untuk mengecek apakah produk merupakan alat kesehatan
def is_medical_device(product_name):
    """Cek apakah produk mengandung kata kunci alat kesehatan"""
    if not isinstance(product_name, str) or product_name.strip() == "":
        return False  # Jika NaN atau kosong, bukan alat kesehatan

    product_name_lower = product_name.lower()

    # Jika mengandung kata kunci alat kesehatan, return True
    return any(keyword in product_name_lower for keyword in medical_device_keywords)


# Terapkan ke dataset
data_ims_prdk['Cleaned Brand Name'] = data_ims_prdk['Product'].apply(clean_product_name)

#Data Survey
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Product Name'].apply(
    lambda x: np.nan if is_medical_device(x) else x
)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(clean_product_name_survey)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].apply(extract_dosage_form)
data_prdk_survey['Cleaned Name'] = data_prdk_survey['Cleaned Name'].str.upper()

#Data PI
data_pi_prdk['Cleaned Name'] = data_pi_prdk['Nama Produk'].apply(extract_dosage_form)

# Standarisasi: Ubah "syringe" menjadi "inj" dalam data IMS
data_ims_prdk["Standardized Name"] = data_ims_prdk["Pack"].str.replace(r"\bsyringe\b", "inj", regex=True)

In [5]:
data_prdk_survey

,Kode,Product Name,Cleaned Name
0,1,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN 20MG TAB/AMAROX
1,2,BISOPROLOL 5 MG /HJ,BISOPROLOL 5 MG /HJ
2,3,BISOPROLOL 5 MG TABLET,BISOPROLOL 5 MG TAB
3,4,CALOS KAPLET,CALOS CAP
4,5,CAPTOPRIL 12.5 MG,CAPTOPRIL 12.5 MG
...,...,...,...
50330,50331,ZYVOX 600MG INJ 300ML,ZYVOX 600MG INJ 300ML
50331,50332,ZYVOX 600MG TAB,ZYVOX 600MG TAB
50332,50333,ZYVOX 600MG/300ML POLYBAG,ZYVOX 600MG/300ML POLYBAG
50333,50334,Zyvox Infusion 2 mg/mL - 300 mL,ZYVOX INF 2 MG/ML - 300 ML


In [ ]:
data_ims_prdk[data_ims_prdk['Product'].str.contains("narfoz", case=False)]

,Company1,Pack,Branded Details,Product,ATC1,ATC2,ATC3,ATC4,Pack Molecule String,Standardized Name,Cleaned Brand Name
15616,PHAROS INDONESIA,NARFOZ AMP 4MG 2ML 5,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ AMP 4MG 2ML 5,narfoz
15617,PHAROS INDONESIA,NARFOZ AMP 8MG 4ML 5,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ AMP 8MG 4ML 5,narfoz
15618,PHAROS INDONESIA,NARFOZ SYR 4MG/5ML 30ML,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ SYR 4MG/5ML 30ML,narfoz
15619,PHAROS INDONESIA,NARFOZ SYR 4MG/5ML 60ML,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ SYR 4MG/5ML 60ML,narfoz
15620,PHAROS INDONESIA,NARFOZ TAB 4MG 12,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ TAB 4MG 12,narfoz
15621,PHAROS INDONESIA,NARFOZ TAB 8MG 12,Branded Generic,NARFOZ - P-I,A - ALIMENTARY T.& METABOLISM,A04 - ANTIEMETICS-ANTINAUSEANTS,A04A - ANTIEMETICS+ANTINAUSEANTS,A04A1 - SEROT ANTAG A-EMET+A-NAUS,ONDANSETRON,NARFOZ TAB 8MG 12,narfoz


In [ ]:
data_prdk_survey[data_prdk_survey['Product Name']=='ABBOTIC 250MG/5ML (50ML)']

,Kode,Product Name,Cleaned Name
131,132,ABBOTIC 250MG/5ML (50ML),ABBOTIC 250MG/5ML 50ML


In [ ]:
data_pi_prdk[data_pi_prdk['Nama Produk'].str.contains('SYRUP', case=False)]

,Item Kode,Nama Produk,Total Penjualan,Cleaned Name
13,008870,NARFOZ 4 MG/5 ML SYRUP 60 ML,4.838020e+09,NARFOZ 4 MG/5 ML SYR 60 ML
16,012697,FIXIPHAR 100 MG/5 ML DRY SYRUP 60 ML,5.770700e+09,FIXIPHAR 100 MG/5 ML DRY SYR 60 ML
30,009496,NYTEX 150 ML DRY SYRUP,2.527080e+08,NYTEX 150 ML DRY SYR
93,012774,OROXIN 1.65/33.3 MG/5 ML SYRUP 60 ML,1.795376e+09,OROXIN 1.65/33.3 MG/5 ML SYR 60 ML
115,025941,NYTEX 100 MG/5 ML DRY SYRUP 75 ML,1.000914e+09,NYTEX 100 MG/5 ML DRY SYR 75 ML
119,012653,LIMOXIN 30 MG/5 ML SYRUP 60 ML,3.514060e+08,LIMOXIN 30 MG/5 ML SYR 60 ML
126,012851,ETHIROS 175 MG/5 ML DRY SYRUP 60 ML,1.093512e+08,ETHIROS 175 MG/5 ML DRY SYR 60 ML
138,015018,NYTEX 100 MG/5 ML DRY SYRUP 75 ML,4.675000e+05,NYTEX 100 MG/5 ML DRY SYR 75 ML
143,019968,PROFED SYRUP 60 ML,6.907260e+08,PROFED SYR 60 ML
155,021827,DELFRIN SYRUP 60 ML,1.400980e+08,DELFRIN SYR 60 ML


# FIXED

## Data IMS and PI Product

In [8]:
# Fungsi untuk mengekstrak angka dan unit dari nama produk
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z]*)', name)  # Tangkap angka + unit
    return {f"{num} {unit}".strip() for num, unit in matches}  # Gunakan set agar unik

# Fungsi untuk mencocokkan angka dengan unit
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0  # Jika salah satu kosong, skor = 0

    survey_set = set(survey_numbers)
    pack_set = set(pack_numbers)
    intersection = survey_set.intersection(pack_set)

    # Skor dihitung berdasarkan banyaknya pasangan angka + satuan yang cocok
    score = (len(intersection) / max(len(survey_set), len(pack_set))) * 100
    return score

# Fungsi untuk mencocokkan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CAP",
                    "CRM", "SUSP", "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    # Temukan bentuk sediaan dalam kedua string
    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    # Jika bentuk sediaan ada di kedua produk, gunakan fuzzy matching
    if survey_form and ims_form:
        return fuzz.partial_ratio(survey_form, ims_form)

    return 50  # Jika tidak ada bentuk sediaan, beri skor default rendah

def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pastikan string

    for pack_tuple in pack_list:
        pack_name, molecule = pack_tuple  # Unpack tuple lebih awal
        pack_numbers = extract_numbers_and_units(pack_name)  # Sekarang ini string

        # Hitung skor kemiripan angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)

        # Hitung skor kemiripan string (nama produk)
        score_strings = fuzz.token_sort_ratio(cleaned_survey_name, pack_name)

        # Hitung kemiripan bentuk sediaan
        score_dosage_form = match_dosage_form(cleaned_survey_name, pack_name)

        # Gabungkan skor dengan bobot (angka lebih penting)
        combined_score = (0.6 * score_numbers) + (0.3 * score_strings) + (0.1 * score_dosage_form)
        results.append((pack_name, molecule, combined_score))

    return results

# Fungsi utama untuk memproses data dengan filter Cleaned Brand
def process_brand_matching(data_ims, data_survey):
    results = []

    # Ambil daftar nilai unik dari 'Cleaned Brand Name' di IMS
    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        # Gunakan regex untuk filter survey yang mengandung brand_keyword
        filtered_survey = data_survey[
            data_survey['Nama Produk'].str.contains(fr"\b{re.escape(brand_keyword)}\b", case=False, na=False)
        ]

        # Ambil hanya produk IMS dengan brand yang sama
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Nama Produk']
            kode_pi = survey_row['Item Kode']

            # Menghitung skor kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Cari produk IMS dengan skor tertinggi
            if similarity_scores:
                best_match_pack, best_molecule, best_score = max(similarity_scores, key=lambda x: x[2])

                # Simpan hasil terbaik
                results.append({
                    'Kode PI' : kode_pi,
                    'Product PI': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    "Pack Molecule String": best_molecule,
                    'Best Similarity Score': best_score
                })

    # Buat DataFrame hasil
    results_df = pd.DataFrame(results)
    return results_df.sort_values(by='Best Similarity Score', ascending=False)

# Contoh penggunaan
filtered_results_pi = process_brand_matching(data_ims_prdk, data_pi_prdk)

filtered_results_pi


,Kode PI,Product PI,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
205,010618,TAXILAN 500MG/5ML SUSPENSI 180ML,TAXILAN SUSP 500MG/5ML 180ML,SUCRALFATE,100.000000
165,004909,POLYSILANE SUSPENSI 100 ML,POLYSILANE SUSP 100ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
167,005085,POLYSILANE SUSPENSI 180 ML,POLYSILANE SUSP 180ML,ALUMINIUM+DIMETICONE+MAGNESIUM,99.302326
140,005866,NOURISH SKIN TABLET @60,NOURISH SKIN TAB 60,ANTHOCYANOSIDE COMPLEX+ASCORBIC ACID+BIOFLAVON...,99.230769
137,005052,NOURISH SKIN TABLET @30,NOURISH SKIN TAB 30,ANTHOCYANOSIDE COMPLEX+ASCORBIC ACID+BIOFLAVON...,99.230769
...,...,...,...,...,...
122,005327,NARFOZ 4 INJEKSI,NARFOZ AMP 4MG 2ML 5,ONDANSETRON,16.875000
39,016888,BINOMIC 10 MG/5 ML SYRUP 60 ML,BINOMIC TAB DISP 20MG 100 BLIS,ZINC,16.551724
198,007054,SCELTO 30 INJEKSI,SCELTO AMP 30MG/ML 1ML 5,KETOROLAC,16.216216
64,012830,DUROCORT 0.25 MG/2 MG TABLET,DUROCORT SYR 60ML,BETAMETHASONE+DEXCHLORPHENIRAMINE,15.714286


In [9]:
filtered_results_pi[filtered_results_pi['Best Similarity Score'] < 60]

,Kode PI,Product PI,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
217,005976,VEGEBLEND 21 JR,VEGEBLEND 21 JR CAP 30,ASCORBIC ACID+BENTONITE+CALCIUM+VEGETABLE FIBR...,59.324324
172,007550,PRAXION 100MG/ML DROPS SUSPENSI,PRAXION DRP 100MG/ML 15ML,PARACETAMOL,59.000000
156,006119,OZEN DROPS 12 ML,OZEN DRP 10MG/ML 12ML,CETIRIZINE,58.857143
87,011520,INHIPUMP 40MG/10ML DRY INJECTION,INHIPUMP VIAL DRY 40MG,OMEPRAZOLE,58.750000
11,020672,MENIN 500 MG FC CAPLET,MENIN CAPLET 500MG 10 10,MEFENAMIC ACID,58.139535
...,...,...,...,...,...
122,005327,NARFOZ 4 INJEKSI,NARFOZ AMP 4MG 2ML 5,ONDANSETRON,16.875000
39,016888,BINOMIC 10 MG/5 ML SYRUP 60 ML,BINOMIC TAB DISP 20MG 100 BLIS,ZINC,16.551724
198,007054,SCELTO 30 INJEKSI,SCELTO AMP 30MG/ML 1ML 5,KETOROLAC,16.216216
64,012830,DUROCORT 0.25 MG/2 MG TABLET,DUROCORT SYR 60ML,BETAMETHASONE+DEXCHLORPHENIRAMINE,15.714286


In [ ]:
hasil = filtered_results_pi.copy()
data_final_pi = data_pi_2024_2025.merge(hasil,
                                        left_on='Item Kode',
                                        right_on='Kode PI',
                                        how='left')
data_final_pi = data_final_pi[['Item Kode','Nama Produk','Best Match IMS Pack','Best Similarity Score','Pack Molecule String','Total Penjualan']]
data_final_pi

,Item Kode,Nama Produk,Best Match IMS Pack,Best Similarity Score,Pack Molecule String,Total Penjualan
0,009100,HERCUM 0.05 % CREAM 10 G,HERCUM CRM 0.05% 10G,95.714286,CLOBETASOL,9.276000e+08
1,013852,ACUCEL 0.1% GEL 10 G,NaN,NaN,NaN,-8.800000e+04
2,008220,IFLACORT CREAM 5 G,IFLACORT CRM 0.1% 5G,63.333333,MOMETASONE,4.093264e+08
3,027173,AYU DERMA WHITENING CARE DAY CREAM 30 G,NaN,NaN,NaN,2.411600e+07
4,029670,NOURISH ACNE PLAST GIRL PROMO BUNDLING,NaN,NaN,NaN,1.018624e+09
...,...,...,...,...,...,...
369,027536,SILKY WHITE WHITENING DAY CREAM 25 G,NaN,NaN,NaN,2.052610e+07
370,012917,QUINOBIOTIC 200 MG/100 ML INFUSION,QUINOBIOTIC TAB 250MG 30,24.245283,CIPROFLOXACIN,7.063620e+08
371,004810,ANTIPRESTIN 20 KAPSUL,ANTIPRESTIN CAP 20MG 30,36.341463,FLUOXETINE,1.201197e+09
372,006108,TRICHOSTATIC OVULA,TRICHOSTATIC VAG SUPP 6,25.487805,METRONIDAZOLE+NYSTATIN,1.179210e+08


### Simpan Hasil ke Gdrive

In [ ]:
import os

file_name = f"27022025 Data Hasil Match Product (PI and IMS).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/27022025 Data Hasil Match Product (PI and IMS).xlsx


In [57]:
data_survey_100 = data_prdk_survey[: 100]

## Data Survey and IMS

In [6]:
# 1️⃣ Fungsi untuk standarisasi nama produk
def standardize_product_name(name):
    if pd.isna(name):
        return ""

    # Hapus tanda kurung kecuali yang mengandung dosis atau satuan
    pattern = r"\((?!\d+\s?(ML|MG|IU|G|MCG|L|KG|M[LI]U|CAP|TAB|VIAL|AMP|PFS|SUP|DRP|TUBE|PATCH|SACH|BAG|BOTTLE))[^)]*\)"
    name = re.sub(pattern, "", name)

    # Hapus karakter khusus kecuali angka, huruf, '%', dan '/'
    name = re.sub(r"[^a-zA-Z0-9\s%/]", "", name)

    return name.strip()

# 2️⃣ Fungsi ekstraksi angka + satuan
def extract_numbers_and_units(name):
    matches = re.findall(r'(\d+\.?\d*)\s?([A-Za-z%/]*)', name)
    return {f"{num} {unit}".strip() for num, unit in matches}

# 3️⃣ Fungsi pencocokan angka + satuan
def match_numbers_with_units(survey_numbers, pack_numbers):
    if not survey_numbers or not pack_numbers:
        return 0

    intersection = survey_numbers.intersection(pack_numbers)
    return (len(intersection) / max(len(survey_numbers), len(pack_numbers))) * 100

# 4️⃣ Fungsi pencocokan bentuk sediaan
def match_dosage_form(survey_str, ims_str):
    common_forms = ["SYRUP", "SYR DRY", "TAB", "CAP", "INJ", "POWDER", "PWD", "SYR", "CRM", "SUSP",
                    "AMP", "OINT", "DRP", "CONC", "SOL", "GEL"]

    survey_form = next((form for form in common_forms if form in survey_str.upper()), "")
    ims_form = next((form for form in common_forms if form in ims_str.upper()), "")

    return fuzz.partial_ratio(survey_form, ims_form) if survey_form and ims_form else 50



# 6️⃣ Fungsi utama untuk menghitung kemiripan
def calculate_similarity(cleaned_survey_name, pack_list):
    results = []
    survey_numbers = extract_numbers_and_units(cleaned_survey_name)  # Pindah ke luar loop

    for pack_tuple in pack_list:
        pack_name, molecule, brand = pack_tuple
        pack_numbers = extract_numbers_and_units(pack_name)

        # Skor angka + satuan
        score_numbers = match_numbers_with_units(survey_numbers, pack_numbers)
        # Skor string berbasis fuzzy matching
        score_strings = fuzz.partial_ratio(cleaned_survey_name, pack_name)
        # Skor bentuk sediaan
        score_dosage_form = 100 if cleaned_survey_name in pack_name else 50  # Lebih sederhana

        # Gabungkan skor tanpa score_brand
        combined_score = (0.4 * score_numbers) + (0.5 * score_strings) + (0.1 * score_dosage_form)

        results.append((pack_name, molecule, brand, combined_score, len(pack_name)))

    # Urutkan berdasarkan skor kemiripan dan panjang nama
    results.sort(key=lambda x: (x[3], x[4]), reverse=True)
    return results


# 7️⃣ Fungsi utama untuk pencocokan produk berdasarkan brand IMS
def process_brand_matching(data_ims, data_survey, threshold=85):
    results = []

    unique_brands = data_ims['Cleaned Brand Name'].dropna().unique()

    for brand_keyword in unique_brands:
        if re.search(r'\bracikan\b|\bracik\b', brand_keyword, re.IGNORECASE):
            continue

        # 💡 Hanya gunakan kata yang bukan angka & lebih dari 3 karakter
        brand_words = [word for word in brand_keyword.lower().split() if len(word) > 3 and not word.isnumeric()]

        # Lewati pencarian jika tidak ada kata tersisa
        if not brand_words:
            continue

        # 💡 **Gunakan RapidFuzz untuk mencocokkan produk survey**
        filtered_survey = filter_survey_by_brand(data_survey, brand_keyword, threshold)

        if filtered_survey.empty:
            continue

        # Ambil hanya produk IMS yang sesuai dengan brand ini
        filtered_ims = data_ims[data_ims['Cleaned Brand Name'] == brand_keyword]
        pack_list = list(zip(filtered_ims['Pack'], filtered_ims['Pack Molecule String'], filtered_ims['Cleaned Brand Name']))

        for _, survey_row in filtered_survey.iterrows():
            cleaned_survey_name = survey_row['Cleaned Name']
            produk_survey = survey_row['Product Name']

            # Hitung kemiripan
            similarity_scores = calculate_similarity(cleaned_survey_name, pack_list)

            # Ambil hasil terbaik
            if similarity_scores:
                best_match = similarity_scores[0]
                best_match_pack, best_molecule, best_brand, best_score, _ = best_match
                results.append({
                    'Product Survey': produk_survey,
                    'Best Match IMS Pack': best_match_pack,
                    'Pack Molecule String': best_molecule,
                    'Brand': best_brand,
                    'Best Similarity Score': best_score
                })

    return pd.DataFrame(results)


# 7️⃣a Fungsi Fuzzy Matching dengan RapidFuzz
def filter_survey_by_brand(data_survey, brand_name, threshold=85):
    brand_name_lower = brand_name.lower()  # Nama brand dari IMS

    def fuzzy_match(survey_name):
        if not isinstance(survey_name, str):
            return False

        # Gunakan RapidFuzz untuk mencari skor kemiripan
        score = fuzz.partial_ratio(brand_name_lower, survey_name.lower())
        return score >= threshold  # Hanya ambil yang skornya di atas threshold

    # **OPTIMASI: Gunakan apply() untuk mengganti looping for**
    return data_survey[data_survey['Cleaned Name'].apply(fuzzy_match)]



# 8️⃣ Eksekusi Program
filtered_results = process_brand_matching(data_ims_prdk, data_prdk_survey)

# Ambil hanya hasil dengan skor tertinggi untuk setiap produk survey
if not filtered_results.empty:
    filtered_results = filtered_results.loc[filtered_results.groupby('Product Survey')['Best Similarity Score'].idxmax()]

filtered_results

,Product Survey,Best Match IMS Pack,Pack Molecule String,Brand,Best Similarity Score
11962,2 KDT RIFAMPICIN/INH 150/75 MG TABLET,RIFAMPICIN CAP 150MG 100,RIFAMPICIN,rifampicin,38.333333
31893,40% DEXTROSE 25 ML^,DEXTROSE INFUSION 5% 500ML,GLUCOSE,dextrose,37.142857
24579,5% DEXTROSE 1/4 NS,DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
24580,5% DEXTROSE 1/4 NS,DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
24581,5% DEXTROSE 1/4 NS ( SANBE ),DEXTROSE + 1/2 NS INFUSION 5% 500ML 20,GLUCOSE+SODIUM,dextrose + 1/2 ns,61.000000
...,...,...,...,...,...
68949,zipren 1 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,43.888889
68951,"zipren 1,5mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,41.842105
68960,"zipren 2,5 mg",ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,40.000000
68962,zipren 2.5 mg,ZIPREN TAB ODT 10MG 10 STRP,ARIPIPRAZOLE,zipren,40.000000


In [7]:
filtered_results[filtered_results['Product Survey'].str.contains('ACETYLCYSTEIN',case=False)]

,Product Survey,Best Match IMS Pack,Pack Molecule String,Brand,Best Similarity Score
18873,ACETYLCYSTEIN 100MG,ACETYLCYSTEINE AMP INHAL 100MG/ML 3ML 10,ACETYLCYSTEINE,acetylcysteine,46.666667
18874,ACETYLCYSTEIN 200 MG CAP,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,65.476190
18875,ACETYLCYSTEIN 200 MG CAPSUL,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,65.476190
18877,ACETYLCYSTEIN 200 MG KAPSUL/MULIA,ACETYLCYSTEINE CAP 200MG 100,ACETYLCYSTEINE,acetylcysteine,65.816327
18878,ACETYLCYSTEIN 200MG CAP,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,66.463415
18879,ACETYLCYSTEIN KAP 200MG,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,72.826087
18880,ACETYLCYSTEIN TAB 200 MG (BPJS),ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,68.750000
18881,ACETYLCYSTEIN TAB 200 MG (FOI),ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,68.750000
18883,ACETYLCYSTEINE 200 MG CAP,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,67.000000
18885,ACETYLCYSTEINE 200 MG CAPSUL,ACETYLCYSTEINE CAP 200MG 100 STRP,ACETYLCYSTEINE,acetylcysteine,67.000000


In [8]:
#Join data hasil match dengan data produk survey (unik)
data_final_survey = data_prdk_survey[['Kode','Product Name']].merge(filtered_results[['Product Survey','Best Match IMS Pack','Pack Molecule String','Best Similarity Score']],
                                                                    left_on='Product Name',
                                                                    right_on='Product Survey',
                                                                    how='left')
data_final_survey = data_final_survey[['Product Name','Best Match IMS Pack','Pack Molecule String','Best Similarity Score']]
data_final_survey

,Product Name,Best Match IMS Pack,Pack Molecule String,Best Similarity Score
0,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN TAB FLM CTD 20MG 100 STRP,ATORVASTATIN,61.363636
1,BISOPROLOL 5 MG /HJ,BISOPROLOL TAB 5MG 30 STRP,BISOPROLOL,64.473684
2,BISOPROLOL 5 MG TABLET,BISOPROLOL TAB FLM CTD 5MG 100 STRP,BISOPROLOL,67.424242
3,CALOS KAPLET,CALOS TAB CHEW 500MG 30,CALCIUM,46.176471
4,CAPTOPRIL 12.5 MG,CAPTOPRIL TAB 12.5MG 100 STRP,CAPTOPRIL,63.235294
...,...,...,...,...
50330,ZYVOX 600MG INJ 300ML,ZYVOX TAB 600MG 10,LINEZOLID,65.000000
50331,ZYVOX 600MG TAB,ZYVOX TAB 600MG 10,LINEZOLID,62.500000
50332,ZYVOX 600MG/300ML POLYBAG,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,59.210526
50333,Zyvox Infusion 2 mg/mL - 300 mL,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,81.538462


### Simpan Hasil ke GDrive

In [9]:
import os

file_name = f"09032025 Data Hasil Match Product (IMS & Survey).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset Hasil Match Product"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_final_survey.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset Hasil Match Product/09032025 Data Hasil Match Product (IMS & Survey).xlsx


# Mapping Dosage Form

## Load Data

In [11]:
# Dapatkan kredensial dan buat klien gspread
creds, _ = default()
gc = gspread.authorize(creds)

# Buka Google Sheet dengan URL atau ID
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1ztO5ypZI3K99wvvTIaUny4PF2VG4tibYSckQR25JPqY'

# Ambil spreadsheet
spreadsheet = gc.open_by_url(spreadsheet_url)

# Pilih worksheet
worksheet = spreadsheet.worksheet('Data Final Sediaan')  # atau gunakan nama worksheet seperti spreadsheet.worksheet('Sheet1')

# Dapatkan data sebagai DataFrame
data_sediaan = get_as_dataframe(worksheet)

In [13]:
# Dapatkan data sebagai DataFrame
data_sediaan = get_as_dataframe(worksheet)

#data_pi = data_final_pi
data_survey = data_final_survey
data_survey_= pd.read_excel('/content/drive/MyDrive/Dataset untuk sistem rekomendasi/06012025 Raw Data Hospital Survey Final.xlsx', sheet_name='Final')

In [14]:
def extract_dosage_form(product_name):
    dosage_form_map = {
        r'\btabs?\b|\btablets?\b|\btablet\b': 'tab',
        r'\bcaps?\b|\bcapsules?\b|\bcapsul\b': 'cap',
        r'\bkapsul\b|\bkapsule\b|\bkap\b': 'cap',
        r'\bsuspensi\b|\bsuspension\b|\bsusp\b': 'susp',
        r'\bserbuk\b|\bbubuk\b|\bpowder\b|\bpwd\b': 'pwd',
        r'\boint\b|\bointment\b|\bsalep\b': 'oint',
        r'\bkaplet\b|\bcaplets?\b|\bkapl\b|\bcaplet\b': 'cap',
        r'\binj\b|\binjeksi\b|\binjection\b': 'inj',
        r'\bamp\b|\bampule\b|\bampul\b': 'amp',
        r'\bcream\b|\bcrm\b|\bkrim\b|\bcr\b': 'crm',
        r'\bsupp\b|\bsuppositoria\b|\bsuppos\b': 'suppos',
        r'\binf\b|\binfusion\b|\binfus\b': 'inf',
        r'\bsyrup\b|\bsirup\b|\bsyr\b': 'syr',
        r'\bdrop\b|\bdrops\b|\btetes\b': 'drp',
        r'\bconc\b|\bconcentrate\b': 'conc',
        r'\bsol\b|\bsolution\b': 'sol',
        r'\bgel\b': 'gel'
    }
    for pattern, standard_form in dosage_form_map.items():
        if re.search(pattern, product_name, flags=re.IGNORECASE):
            return standard_form
    return ''

In [16]:
# Mapping bentuk sediaaan data PI
#data_pi['Dosage Form'] = data_pi['Nama Produk'].apply(extract_dosage_form)

# Mapping bentuk sediaaan data Survey
data_survey['Dosage Form'] = data_survey['Product Name'].apply(extract_dosage_form)

In [ ]:
#merged_df_pi = pd.merge(data_pi, data_sediaan, left_on='Dosage Form', right_on='Form', how='left')
#merged_df_pi.drop(columns=['Form'], inplace=True)
#merged_df_pi

,Item Kode,Nama Produk,Best Match IMS Pack,Best Similarity Score,Pack Molecule String,Total Penjualan,Dosage Form,Kode Form,Jenis Form
0,009100,HERCUM 0.05 % CREAM 10 G,HERCUM CRM 0.05% 10G,95.714286,CLOBETASOL,9.276000e+08,crm,G,TOPIC DERMATOL HAEMOR EXTERN
1,013852,ACUCEL 0.1% GEL 10 G,NaN,NaN,NaN,-8.800000e+04,gel,F,Oral Topical
2,013852,ACUCEL 0.1% GEL 10 G,NaN,NaN,NaN,-8.800000e+04,gel,G,TOPIC DERMATOL HAEMOR EXTERN
3,008220,IFLACORT CREAM 5 G,IFLACORT CRM 0.1% 5G,63.333333,MOMETASONE,4.093264e+08,crm,G,TOPIC DERMATOL HAEMOR EXTERN
4,027173,AYU DERMA WHITENING CARE DAY CREAM 30 G,NaN,NaN,NaN,2.411600e+07,crm,G,TOPIC DERMATOL HAEMOR EXTERN
...,...,...,...,...,...,...,...,...,...
382,027536,SILKY WHITE WHITENING DAY CREAM 25 G,NaN,NaN,NaN,2.052610e+07,crm,G,TOPIC DERMATOL HAEMOR EXTERN
383,012917,QUINOBIOTIC 200 MG/100 ML INFUSION,QUINOBIOTIC TAB 250MG 30,24.245283,CIPROFLOXACIN,7.063620e+08,inf,C,Parenteral
384,004810,ANTIPRESTIN 20 KAPSUL,ANTIPRESTIN CAP 20MG 30,36.341463,FLUOXETINE,1.201197e+09,cap,A,Oral Solid
385,006108,TRICHOSTATIC OVULA,TRICHOSTATIC VAG SUPP 6,25.487805,METRONIDAZOLE+NYSTATIN,1.179210e+08,,NaN,NaN


In [17]:
# Mapping bentuk sediaaan data Survey
merged_df_survey = pd.merge(data_survey, data_sediaan, left_on='Dosage Form', right_on='Form', how='left')
merged_df_survey.drop(columns=['Form'], inplace=True)
merged_df_survey

,Product Name,Best Match IMS Pack,Pack Molecule String,Best Similarity Score,Dosage Form,Kode Form,Jenis Form
0,ATORVASTATIN 20MG TAB/AMAROX,ATORVASTATIN TAB FLM CTD 20MG 100 STRP,ATORVASTATIN,61.363636,tab,A,Oral Solid
1,BISOPROLOL 5 MG /HJ,BISOPROLOL TAB 5MG 30 STRP,BISOPROLOL,64.473684,,NaN,NaN
2,BISOPROLOL 5 MG TABLET,BISOPROLOL TAB FLM CTD 5MG 100 STRP,BISOPROLOL,67.424242,tab,A,Oral Solid
3,CALOS KAPLET,CALOS TAB CHEW 500MG 30,CALCIUM,46.176471,cap,A,Oral Solid
4,CAPTOPRIL 12.5 MG,CAPTOPRIL TAB 12.5MG 100 STRP,CAPTOPRIL,63.235294,,NaN,NaN
...,...,...,...,...,...,...,...
50685,ZYVOX 600MG INJ 300ML,ZYVOX TAB 600MG 10,LINEZOLID,65.000000,inj,C,Parenteral
50686,ZYVOX 600MG TAB,ZYVOX TAB 600MG 10,LINEZOLID,62.500000,tab,A,Oral Solid
50687,ZYVOX 600MG/300ML POLYBAG,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,59.210526,,NaN,NaN
50688,Zyvox Infusion 2 mg/mL - 300 mL,ZYVOX POLYBAG 2MG/ML 300ML,LINEZOLID,81.538462,inf,C,Parenteral


In [18]:
merged_df_survey = merged_df_survey.drop_duplicates('Product Name')
data_survey_merged = pd.merge(data_survey_, merged_df_survey[['Product Name','Pack Molecule String','Dosage Form']],
                              left_on='Product Name without #',
                              right_on='Product Name',
                              how='left')
data_survey_merged

,Tahun,Hospital Name,Doctor Name,Product Name without #,1,2,3,4,5,6,7,8,9,10,11,12,Product Name,Pack Molecule String,Dosage Form
0,2020,RS Borromeus Bandung,Ahli Gizi Klinik,NUTREN DIABETES VANILA 400G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,NUTREN DIABETES VANILA 400G,NALTREXONE,
1,2020,RS Borromeus Bandung,Ahli Gizi Klinik,PROTEN GOLD (VANILA),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,PROTEN GOLD (VANILA),BIFIDOBACTERIUM LONGUM+FRUCTOOLIGOSACCHARIDES+...,
2,2020,RS Borromeus Bandung,APOTEKER,ABILIFY 5 MG TAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,0.0,0.0,0.0,30.0,ABILIFY 5 MG TAB,ARIPIPRAZOLE,tab
3,2020,RS Borromeus Bandung,APOTEKER,AC SWAB 2% @100,15.0,80.0,0.0,0.0,35.0,0.0,0.0,0.0,50.0,20.0,0.0,486.0,AC SWAB 2% @100,NaN,
4,2020,RS Borromeus Bandung,APOTEKER,ACANTHE SUNSCREEN SPF 30 CREAM 30GR,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,6.0,ACANTHE SUNSCREEN SPF 30 CREAM 30GR,ASCORBIC ACID+AVOBENZONE+CYCLOPENTASILOXANE+OC...,crm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
659164,2024,RS Prima Medika,RSPM,ZOTER CREAM 5GR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,ZOTER CREAM 5GR,ACICLOVIR,crm
659165,2024,RS Prima Medika,RSPM,ZYCIN 500 MG TABLET,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,9.0,NaN,NaN,NaN,ZYCIN 500 MG TABLET,AZITHROMYCIN,tab
659166,2024,RS Prima Medika,RSPM,"ZYPRAZ 0,5 MG TAB",NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,"ZYPRAZ 0,5 MG TAB",ALPRAZOLAM,tab
659167,2024,RS Prima Medika,Tias Puspaningrum Amd Kep,ACETIN 200 MG KAP,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ACETIN 200 MG KAP,ACETYLCYSTEINE,cap


## Simpan ke GDrive

In [19]:
import os

file_name = f"Data Produk Survey (Final).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset untuk sistem rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
data_survey_merged.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk Survey (Final).xlsx


In [ ]:
file_name = f"Data Produk PI (Final).xlsx"  # Format nama file

# Tentukan direktori penyimpanan
save_directory = "/content/drive/MyDrive/Dataset untuk sistem rekomendasi"
file_path = os.path.join(save_directory, file_name)

# Simpan DataFrame ke file Excel
merged_df_pi.to_excel(file_path, index=False)

print(f"File berhasil disimpan di: {file_path}")

File berhasil disimpan di: /content/drive/MyDrive/Dataset untuk sistem rekomendasi/Data Produk PI (Final).xlsx
